# DBL Data Challenge
Team Members: Atharva Kale, Borna Bengez, Donghyeok Lee, Jona Loxha, Paula Dodig, Philipp Merz, Swaraj Shah<br>
WRITE UP SHORT SUMMARY

In [1]:
#import important modules
import pymongo as pm
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn as skl
from pymongo import MongoClient
import secrets
import json

In [14]:
with open('editeddata/conversations.json', 'r') as convo_json:
    convos = json.load(convo_json)

In [2]:
client = MongoClient('localhost', 27017)

In [3]:
mydb = client.tweets

In [4]:
col = mydb['_tweets']

In [82]:
import pprint
pprint.pprint(col.find_one({'language':'eng'}))

{'_id': ObjectId('5eb6ce36836b298628c00c01'),
 'id': 1131173065122078700,
 'text': 'So @AmericanAir @EWRairport lied to an old lady causing her to miss her flight #934 to LHR. She arrived at 6:05am for a flight departing at 8:30am,mum has no medication,is in a wheelchair and they just ignore the fact they caused this.Worse airline company. Zero customer service',
 'in_reply_to_status_id': None,
 'in_reply_to_user_id': None,
 'user': {'id': 2730172562,
  'followers_count': 163,
  'verified': False,
  'location': 'A rock in space'},
 'contributors': None,
 'is_quote_status': False,
 'quote_count': 0,
 'reply_count': 0,
 'retweet_count': 0,
 'favorite_count': 0,
 'timestamp_ms': '1558527649561',
 'language': 'eng'}

In [41]:
#all conversations from conversations.json
nr_conv = 0
for key in convos:
    nr_conv += 1
nr_conv

135572

In [89]:
#all tweets from tweets.json
nr_all = 0
for t in col.find():
    nr_all += 1
nr_all

1531145

Airlines and their IDs

In [18]:
airlines = {'KLM':56377143, 
            'AirFrance': 106062176, 
            'British_Airways': 18332190, 
            'AmericanAir': 22536055, 
           'Lufthansa': 124476322, 
           'AirBerlin': 26223583, 
           'AirBerlinAssist': 2182373406, 
           'easyJet': 38676903, 
           'RyanAir': 1542862735, 
           'SingaporeAir': 253340062, 
           'Qantas': 218730857, 
           'EtihadAirways': 45621423, 
           'VirginAtlantic': 20626359}

Connect the tweets to airlines and store the total length of all posts for an airline 

In [39]:
air_count = {}
for name in airlines:
    air_count[name] = {}
    air_count[name]['nr'] = 0
    air_count[name]['len'] = 0
    for tweet in col.find():
        if name in tweet['text'] or airlines[name] == tweet['in_reply_to_user_id'] or airlines[name] == tweet['user']['id']:
            air_count[name]['nr'] += 1
            air_count[name]['len'] += len(tweet['text'].split())

Get the average length of a post (in words)

In [58]:
for name in air_count:
    if air_count[name]['nr'] != 0:
        air_count[name]['avg_post'] = round(air_count[name]['len']/air_count[name]['nr'])
    else:
        air_count[name]['avg_post'] = 0
air_count

{'KLM': {'nr': 152535, 'len': 5334608, 'avg_post': 35},
 'AirFrance': {'nr': 37615, 'len': 1324758, 'avg_post': 35},
 'British_Airways': {'nr': 266463, 'len': 9695674, 'avg_post': 36},
 'AmericanAir': {'nr': 292715, 'len': 10546149, 'avg_post': 36},
 'Lufthansa': {'nr': 90561, 'len': 2972093, 'avg_post': 33},
 'AirBerlin': {'nr': 416, 'len': 13273, 'avg_post': 32},
 'AirBerlinAssist': {'nr': 0, 'len': 0, 'avg_post': 0},
 'easyJet': {'nr': 169221, 'len': 6150931, 'avg_post': 36},
 'RyanAir': {'nr': 48714, 'len': 1789020, 'avg_post': 37},
 'SingaporeAir': {'nr': 29051, 'len': 1013149, 'avg_post': 35},
 'Qantas': {'nr': 92245, 'len': 3209933, 'avg_post': 35},
 'EtihadAirways': {'nr': 19163, 'len': 625183, 'avg_post': 33},
 'VirginAtlantic': {'nr': 64985, 'len': 2267713, 'avg_post': 35}}

In [75]:
df = pd.DataFrame(columns=('airline', 'nr_tweets', 'avg_tweet'))

In [76]:
a = []
b = []
for name in air_count: 
    a.append(air_count[name]['nr'])
    b.append(air_count[name]['avg_post'])

In [77]:
df['airline'] = ['KLM', 'AirFrance', 'British_Airways', 'AmericanAir', 'Lufthansa', 'AirBerlin', 'AirBerlinAssist', 'easyJet', 'RyanAir', 'SingaporeAir', 'Qantas', 'EtihadAirways', 'VirginAtlantic']
df['nr_tweets'] = a
df['avg_tweet'] = b
df

,airline,nr_tweets,avg_tweet
0,KLM,152535,35
1,AirFrance,37615,35
2,British_Airways,266463,36
3,AmericanAir,292715,36
4,Lufthansa,90561,33
5,AirBerlin,416,32
6,AirBerlinAssist,0,0
7,easyJet,169221,36
8,RyanAir,48714,37
9,SingaporeAir,29051,35


In [67]:
nr_replies = []
for con in convos:
    nr_replies.append(len(convos[con]['convs']))

In [69]:
#average number of replies a tweet gets 
sum(nr_replies)/len(nr_replies)

1.6757147493582745

In [110]:
#users_influencial = 0
#tweets_per_infl_user = []
#for tweet in col.find({'user':{'followers_count': {'$gt' : 100}}}):
 #   users_influential += 1
  #  for t in col.find():
   #     if t['user'] == tweet['user']:
    #        nr_tweets_of_the_guy += 1
    #tweets_per_infl_user.append(nr_tweets_of_the_guy)

In [111]:
#users_influencial
#tweets_per_infl_user

In [112]:
#c_per_air = {}
#for name in airlines:
 #   c_per_air[name] = 0
  #  for c in convos:
   #     k = col.find_one({'id': int(c)})
    #    if name in k['text'] or airlines[name] == k['in_reply_to_user_id'] or airlines[name] == k['user']['id']:
     #       c_per_air[name] += 1